In [1]:
import numpy as np
from tqdm import tqdm

def get_sic_vertices(d: int) -> np.ndarray:
    """
    Constructs the matrix whose columns are the SIC simplex vertices.

    Args:
        d: The dimension of the Hilbert space.

    Returns:
        A (d^2, d^2) numpy array where each column is a SIC vertex.
    """
    n = d**2
    vertices = np.zeros((n, n))
    
    # The term (1 / (d * (d + 1))) is common to all elements
    common_factor = 1.0 / (d * (d + 1))
    
    # Create the matrix of all ones
    all_ones = np.ones((n, n))
    
    # Create the scaled identity matrix for the delta term
    identity_term = d * np.identity(n)
    
    # The vertices are the columns of the resulting matrix
    vertices = common_factor * (identity_term + all_ones)
    
    return vertices

def is_in_convex_hull(p: np.ndarray, V_inv: np.ndarray) -> bool:
    """
    Checks if a point p is inside the convex hull of the vertices V.

    A point is inside the convex hull if it can be written as a convex
    combination of the vertices, which means the coefficients (alpha)
    solved from p = V * alpha are all non-negative.

    Args:
        p: The random point to check.
        V_inv: The pre-calculated inverse of the vertex matrix.

    Returns:
        True if the point is inside the hull, False otherwise.
    """
    # Solve the system V * alpha = p for alpha
    alpha = V_inv @ p
    
    # Check if all coefficients are non-negative.
    # A small tolerance is used to account for floating-point inaccuracies.
    return np.all(alpha >= -1e-9)

def estimate_volume_ratio(d: int, num_samples: int) -> float:
    """
    Estimates the ratio of the SIC simplex volume to the probability simplex volume
    using a Monte Carlo method.

    Args:
        d: The dimension of the Hilbert space.
        num_samples: The number of random points to generate for the simulation.

    Returns:
        The estimated volume ratio.
    """
    n = d**2
    
    print(f"Setting up for d={d} (n=d^2={n})...")
    
    # 1. Get the SIC vertices and pre-calculate the inverse of the vertex matrix.
    # This is a significant optimization as the inverse is constant for all samples.
    V = get_sic_vertices(d)
    try:
        V_inv = np.linalg.inv(V)
    except np.linalg.LinAlgError:
        print("Error: The vertex matrix is singular and cannot be inverted.")
        return 0.0

    # 2. Run the Monte Carlo simulation.
    inside_count = 0
    print(f"Running simulation with {num_samples:,} samples...")
    
    for _ in tqdm(range(num_samples), desc="Monte Carlo Progress"):
        # Generate a random point uniformly distributed in the probability simplex.
        # This is done by sampling from an exponential distribution and normalizing.
        random_exponentials = np.random.exponential(1.0, size=n)
        p = random_exponentials / np.sum(random_exponentials)
        
        # Check if the point lies within the SIC simplex (the convex hull of V).
        if is_in_convex_hull(p, V_inv):
            inside_count += 1
            
    # 3. The estimated ratio is the fraction of points that fell inside.
    return inside_count / num_samples

def main():
    """
    Main function to run the simulation and display results.
    """
    # --- Simulation Parameters ---
    d = 2  # Dimension. Note: d > 2 is computationally infeasible.
    num_samples = 2_000_000  # Number of Monte Carlo samples. More is better.
    
    if d > 2:
        print(f"Warning: Monte Carlo estimation for d={d} is not practical.")
        print("The true volume ratio is extremely small, and an enormous number")
        print("of samples would be needed to get a meaningful result.\n")

    # --- Run Simulation ---
    estimated_ratio = estimate_volume_ratio(d, num_samples)
    
    # --- Calculate Analytical Result ---
    true_ratio = (1 / (d + 1))**(d**2 - 1)
    
    # --- Display Results ---
    print("\n--- Results ---")
    print(f"Dimension (d): {d}")
    print(f"Number of samples: {num_samples:,}")
    print("-" * 15)
    print(f"Monte Carlo Estimated Ratio: {estimated_ratio:.8f}")
    print(f"Analytical (True) Ratio:    {true_ratio:.8f}")
    print(f"Absolute Error:             {abs(estimated_ratio - true_ratio):.8f}")

if __name__ == "__main__":
    main()


Setting up for d=2 (n=d^2=4)...
Running simulation with 2,000,000 samples...


Monte Carlo Progress: 100%|██████████| 2000000/2000000 [00:32<00:00, 61248.35it/s]


--- Results ---
Dimension (d): 2
Number of samples: 2,000,000
---------------
Monte Carlo Estimated Ratio: 0.03736450
Analytical (True) Ratio:    0.03703704
Absolute Error:             0.00032746


In [7]:
import math

d = 2
d/math.factorial(d**2-1)

0.3333333333333333

In [8]:
get_sic_vertices(2)

array([[0.5       , 0.16666667, 0.16666667, 0.16666667],
       [0.16666667, 0.5       , 0.16666667, 0.16666667],
       [0.16666667, 0.16666667, 0.5       , 0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.5       ]])

In [1]:
import numpy as np

In [3]:
H = np.random.randn(10, 4); H

array([[ 0.1316285 , -0.08525514, -0.64936486, -0.33787555],
       [-0.30212997, -0.97877979,  0.75209679,  2.20162709],
       [ 0.38275558,  0.77453701,  1.11665022, -0.50611898],
       [-1.34193137,  1.26599323,  0.20879645,  0.92909578],
       [-0.80640376, -0.20585447, -0.52628588,  0.55416066],
       [-0.367277  ,  0.56016837, -1.40588212, -0.01163235],
       [ 0.52771807,  1.36586582, -0.40366818, -2.53118717],
       [ 1.84544533, -0.10004747,  0.82872325,  1.43425753],
       [-0.62156097,  1.17884826,  1.03757362, -1.75852017],
       [ 0.87154209, -0.03563278,  0.66069258,  0.44346111]])

In [ ]:
np.linalg.